In [ ]:
import os
os.chdir("project_dir")
os.listdir()

In [ ]:
import pickle
import tensorflow as tf
import datetime
import pandas as pd
import random
import glob
import os
from itertools import islice
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import math

In [ ]:
glob.glob('project_dir/data/'+'*csv')

# Functions

In [ ]:
def SQSumSq(df):
    # compute Signal Vector Magnitude per input
    return (df**2).sum(axis=1)**0.5

In [ ]:
def read_data(folder1 = '... /Energy_Expenditure_Measurements/'):

    ankle = ['ankle_x', 'ankle_y', 'ankle_z']
    wrist = ['wrist_x', 'wrist_y', 'wrist_z'] 

    df_gene = pd.DataFrame()
    df_cos = pd.DataFrame()

    print('loading all datafiles...')
    for data in glob.glob(folder1+'*csv'):
        data = os.path.splitext(os.path.basename(data))[0]
        # skip participants with no cosmoed or some accel are missing
        if data in ['GOTOV02', 'GOTOV03', 'GOTOV04', 'GOTOV19']:
                  # ,'GOTOV12',GOTOV16','GOTOV18', 'GOTOV23', 'GOTOV27','GOTOV30']:
            continue
        else:
#           # read data
            df = pd.read_csv(folder1+data+".csv", header = 0, index_col = None, low_memory=False)
            # keep target data and their timestamps
            df_cosmed = df[['EEm', 'time']]
            df_cosmed.rename(columns={'time':'time_cosmed'})
            # keep timestamps separately
            time = df.time

            # keep accelerometer data
            df_ankle = df[ankle]
            # df_ankle = SQSumSq(df_ankle) # this can be used if you want to train with SVMs instead of all x,y,z
            df_wrist = df[wrist]
            # df_wrist = SQSumSq(df_wrist) # this can be used if you want to train with SVMs instead of all x,y,z
            df = pd.concat([time, df_ankle, df_wrist], axis=1)
            # keep predicted activities
            predAR = df[['time','predicted_activity_label']]                    
            predAR = predAR[['time','label']]

            # concat with activities
            df = df.merge(predAR, left_on='time', right_on='time', how='left')
            df['label'] = df['label'].fillna(method='ffill')
            df['label'] = df['label'].fillna(method='bfill')

    #         print(df)
            inv_yhat = np.empty((df.shape[0], 2))
            inv_yhat.fill(np.nan)
            inv_yhat[:df_cosmed.shape[0]] = df_cosmed
            df_cosmed = pd.DataFrame(inv_yhat, columns=['EEm','time_cos'])
    #         print(df_cosmed)

            df = pd.concat([df, df_cosmed], axis=1)
            df['participant'] = data

            df_gene = df_gene.append(df)
        #fi
    #efor
    return df_gene
#edef

In [ ]:
# select data
def select_Data(name, randomSeed, numberForVal, df_gene):
###### select datafiles. 
# This function creates dataFrames for train, test and validation datasets for the LOSO-CV.
# The read_data function takes two variables, i.e, the name of the participant ('GOTOV05) and their directory. 
# Then it creates the validation set which contains 2 participants selected at random (one with all data, one with
# only indoors data). The random.seed is fixed always to the number of the participants selected for reproducable
# results. The participant name passed as a variable above is used to create the test set.
# The rest are used in training the model. 

    # name: the test participant
    # randomSeed: as random seed we select the number of participants' id
    # numberForVal: the number of participants you want per group (all_data/only_indoors) as validation set 
    # df_gene: data export from read_data
######      
    df_train, df_val, df_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    print(name)
    random.seed(randomSeed)
    print('Random seed is:',randomSeed)

    data_to_select_random_ = list(df_gene['participant'].unique())

    # exclude participant for test and build validation set
    if name in data_to_select_random_: data_to_select_random_.remove(name) #fi
  
    # set of participants with all data
    all_act_data = ['GOTOV08', 'GOTOV10', 'GOTOV11', 'GOTOV12', 
                   'GOTOV17', 'GOTOV20', 'GOTOV21', 'GOTOV28',
                   'GOTOV29', 'GOTOV31', 'GOTOV33', 'GOTOV35']
    # exclude participant for test if in all_data
    if name in all_act_data: all_act_data.remove(name) #fi

    # set of participants with only indoors activities
    indoors_act_data = ['GOTOV22', 'GOTOV23', 'GOTOV13','GOTOV14', 
                       'GOTOV24', 'GOTOV25', 'GOTOV26', 'GOTOV27', 
                       'GOTOV30', 'GOTOV32', 'GOTOV34', 'GOTOV36']
    # exclude participant for test if in indoors_act_data
    if name in indoors_act_data: indoors_act_data.remove(name) #fi
    # select partcipants for val_set
    validation_data = [random.sample(all_act_data, numberForVal)[0], 
                       random.sample(indoors_act_data, numberForVal)[0]]

    print('Getting val and train data.....')
    # build train, val and test sets
    for data in data_to_select_random_: 
        if data in validation_data:
            df_val = df_val.append(df_gene.query('participant == "'+data+'"'))
            print('val_data:', data)
        else:
            df_train = df_train.append(df_gene.query('participant == "'+data+'"'))
        #fi
    #for
    print('Getting test data.....')
    df_test = df_test.append(df_gene.query('participant == "'+name+'"'))
    print('Done creating all dataframes.....')
    
    return df_train, df_val, df_test
#edef

In [ ]:
def standardizing_data(name, Xtrain, Xval, Xtest, SEQUENCE_SIZE, downsample_rate, func, EEm_rate, save_dir):
###### Standarize data for training
# This function takes as an input the data frames created by read_data() function and uses the StandardScaler() to
# scale them in order to create the training sequences needed.
    # name: the test participant
    # Xtrain: the train dataset
    # Xval: the val dataset
    # Xtest: the test dataset   
    
    # the rest of the inputs are passed as input to the  sequence_builder function
    # SEQUENCE_SIZE: the length of the sequences to be build
        # This input is given in terms of instance, e.g. for a 4min window n= 4*60*83, where 4 is the minutes, 
        # and 83 the Sampling Rate
        # For example 465 is the number of instances for a window of 4 minutes with a sampling rate of SR=2Hz, 
        # downsampled from orignal data. If the SR is changed it has to be recalculated. 
        # For more details about the sequences see below
    # downsample_rate: the sampling rate to downsample the accelerometer data for the sequences
    # func: is the function used for sampling rate
        # 'mean': keep the mean of the window selected for downsampling
        # 'std': keep the standard deviation of the window selected for downsampling
        # 'qdif': keep the given percentiles difference (0.05 and 0.95 for us here) of the window selected for downsampling
        # 'iqr_dif': keep the interquartile range of the window selected for downsampling        
    # EEm_rate: the downsampled rate of EEm values. Since COSMED give data per breath (unstable rate) we need to 
    #           fix the EEm rate to be stable.  EEm_rate == 0  no downsampling of target var EEm.
    # save_dir: the directory where the sequences are saved
######     
    print('Scaling data....')

    # variables to drop and use as columns later 
    cols = ['time','label','time_cos', 'participant']

    cols_df = Xtrain.columns.tolist()
    cols_df = cols_df[1:] + cols_df[:1]
    cols_act = ['time','label']

    # scaling accelerometer measurements per set (train, val, test)
    X_train = Xtrain.drop(cols, axis=1)
    y_train = Xtrain[cols].values

    X_val = Xval.drop(cols,  axis=1)
    y_val = Xval[cols].values

    X_test = Xtest.drop(cols,  axis=1)
    y_test = Xtest[cols].values
    
    # load scaler
    scaler = StandardScaler()
    # with mask we avoid the na's while standarizing
    X_train = X_train.values.astype('float32')
    X_train = np.ma.array(X_train, mask=np.isnan(X_train))
    # fit scaler to train set
    scaler.fit(X_train)
    #scale train set
    X_train = scaler.transform(X_train)
    #scale val set
    X_val = X_val.values.astype('float32')
    X_val = np.ma.array(X_val, mask=np.isnan(X_val))
    X_val = scaler.transform(X_val)
    #scale test set
    X_test = X_test.values.astype('float32')
    X_test = np.ma.array(X_test, mask=np.isnan(X_test))
    X_test = scaler.transform(X_test)

    #concat again predictors with train data
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)
    X_train = pd.concat([X_train, y_train], axis=1)
    X_train.columns = cols_df + cols

    X_val = pd.DataFrame(X_val)
    y_val = pd.DataFrame(y_val)
    X_val = pd.concat([X_val, y_val], axis=1)
    X_val.columns = cols_df + cols

    X_test = pd.DataFrame(X_test)
    y_test = pd.DataFrame(y_test)
    X_test = pd.concat([X_test, y_test], axis=1)
    X_test.columns = cols_df + cols

    # one hard encoder for nominal data
    encoder = LabelEncoder()
    X_train.label = encoder.fit_transform(X_train.label)
    X_val.label =  encoder.fit(X_val.label) 
    X_test.label =  encoder.fit(X_test.label) 

    #read, add and standarize participant level data (age, sex, weight, height)
    encoder = LabelEncoder()

    part_level_data_file = pd.read_csv('GOTOV_DataSummary.csv')
    part_level_data_file_cols = ['ID', 'gender', 'age', 'weight', 'height', 'bmi']
    part_level_data_file = part_level_data_file[part_level_data_file_cols]
    part_level_data_file.gender = encoder.fit_transform(part_level_data_file.gender)

    part_level_data_columns = part_level_data_file.columns.tolist()
    part_level_data_columns =  part_level_data_columns[2:]+ part_level_data_columns[:-4]
    part_level_data_file_numeric_cols = ['age', 'weight', 'height', 'bmi']
    age_wt_ht = part_level_data_file[part_level_data_file_numeric_cols].values
    part_sex = part_level_data_file.gender.values 

    scaler2 = StandardScaler()
    age_wt_ht = scaler2.fit_transform(age_wt_ht)

    age_wt_ht = pd.DataFrame(age_wt_ht)
    part_sex = pd.DataFrame(part_sex)
    part_level_data_file = pd.concat([age_wt_ht, part_sex], axis=1)
    part_level_data_file.columns = part_level_data_columns

    print('Done scaling data....')

    # Call functions to build sequences
    print('Building Train sequences.....')
    X_train, y_train, ytrain_time, bmi_train = sequence_builder(X_train, part_level_data_file, SEQUENCE_SIZE, downsample_rate, func, EEm_rate)
    print('Building Val sequences.....')
    X_val, y_val, yval_time, bmi_val = sequence_builder(X_val, part_level_data_file, SEQUENCE_SIZE, downsample_rate, func, EEm_rate)
    print('Building Test sequences.....')
    X_test, y_test, ytest_time, bmi_test = sequence_builder(X_test, part_level_data_file, SEQUENCE_SIZE, downsample_rate, func, EEm_rate)

    print('Saving sequences.... ')
    with open(save_dir+name+'.pkl','wb') as f:
        pickle.dump((X_train, y_train, ytrain_time, bmi_train, 
                     X_val, y_val, yval_time, bmi_val, 
                     X_test, y_test, ytest_time, bmi_test, 
                     scaler), f)

In [ ]:
# Custom downsampling functions added to the mean and std
def dif_quantiles(df, limits = [.05, .95]):
####
# keep the given percentiles difference (0.05 and 0.95 for us here) of the df
####
    q = list(df.quantile(limits))
#     diff_q = np.abs(q[1]-q[0])
    diff_q = q[1]-q[0]
    return diff_q

def iqr_dif(df, limits = [.25, .75]):
####
# keep the interquartile range of the df
####
    q = list(df.quantile(limits))
#     diff_q = np.abs(q[1]-q[0])
    diff_q = q[1]-q[0]
    return diff_q


In [ ]:
def sequence_builder(input_dataset, part_level_data, SEQUENCE_SIZE, downsample_rate, func, EEm_rate):
###### 
# The sequence_builder function accepts a dataFrame returned by the standardizing_data function above and 
# transforms the data inputs into sequences required for training the machine learning model. 
# The to_sequences function (to_sequences_data()) is called within the sequence_builder function. 
    # input_dataset: the standardised input dataset with predictors and target data
    # part_level_data: the standardised participant level data
    # SEQUENCE_SIZE: the length of the sequences to be build
        # This input is given in terms of instance, e.g. for a 4min window n= 4*60*83, where 4 is the minutes, 
        # and 83 the Sampling Rate. For example 465 is the number of instances for a window of 4 minutes with 
        # a sampling rate of SR=2Hz, downsampled from orignal data. If the SR is changed it has to be recalculated. 
    # downsample_rate: the sampling rate to downsample the accelerometer data for the sequences
    # func: is the function used for sampling rate
        # 'mean': keep the mean of the window selected for downsampling
        # 'std': keep the standard deviation of the window selected for downsampling
        # 'qdif': keep the given percentiles difference (0.05 and 0.95 for us here) of the window selected for downsampling
        # 'iqr_dif': keep the interquartile range of the window selected for downsampling        
    # EEm_rate: the downsampled rate of EEm values. Since COSMED give data per breath (unstable rate) we need to 
    #           fix the EEm rate to be stable. EEm_rate == 0  no downsampling of target var EEm.
###### 
    #initialize
    seqX, seqY, seqYtime, seqB = [], [], [], []
    cols = ['time_cos', 'EEm']
    k=1
    for data in input_dataset['participant'].unique(): 
        # print('building sequence', k,'/', df_train['participant'].nunique(), ', participant - ',data)
        k = k+1
        df_input_dataset = input_dataset.query('participant == "'+data+'"')

        part_level_data_part = part_level_data[part_level_data.participant==data]
        part_level_data_part = part_level_data_part.drop('participant', axis=1).values.tolist()

        df_accelData = df_input_dataset.drop(cols, axis=1)
        df_cosmed = df_input_dataset[cols]

        df_accelData.set_index('time', inplace=True)
        df_accelData.index = pd.to_datetime(df_accelData.index, unit = "ms")

        df_cosmed.set_index('time_cos', inplace=True)
        df_cosmed.index = pd.to_datetime(df_cosmed.index, unit = "ms")

        labels = df_accelData['label']
        
        x_values = df_accelData.drop(['participant','label'], axis=1).sort_index()
        idx = df_cosmed['EEm'].notnull()
        y_values = df_cosmed['EEm'][idx].sort_index()

        # downsampling accelerometer
        key = str(round((1/downsample_rate), 3)) + 'S' # sampling rate for downsampling
        if func == 'mean':
            x_values = x_values.resample(key).mean()
        elif func == 'std':
            x_values = x_values.resample(key).std()
        elif func == 'qdif':
            x_values = x_values.resample(key).apply(dif_quantiles)
        elif func == 'iqr_dif':
            x_values = x_values.resample(key).apply(dif_quantiles)                    
             
        # delete created na values between indoors and outdoors data
        idx1 = x_values['ankle_x'].notnull()
        x_values = x_values[idx1]

        x_values = pd.merge(x_values, labels, left_index=True, right_index=True, how = 'outer')
        x_values['label'] = x_values['label'].fillna(method='ffill')
        x_values['label'] = x_values['label'].fillna(method='bfill')
        idx1 = x_values['ankle_x'].notnull()
        x_values = x_values[idx1]

        # downsampling EEm
        if EEm_rate == 0: # no downsampling of target var EEm
            y_values = df_cosmed['EEm'][idx].sort_index()
        else:
            key2 = str(EEm_rate) + 'S' # sampling rate for downsampling
            y_values = y_values.resample(key2).mean()
            idx2 = y_values.notnull()
            y_values = y_values[idx2]
        #fi
        
        # create sequences per window of data using function to_sequences_data()
        x, y, bmi, y_time = to_sequences_data(SEQUENCE_SIZE, x_values, y_values, part_level_data_part, downsample_rate)
        seqX.append(x)
        seqY.append(y)
        seqYtime.append(y_time)
        seqB.append(bmi)
    #efor
    
    # reshaping sequences  
    x = np.vstack(seqX)
    bmi = np.vstack(seqB)
    y = [item for sublist in seqY for item in sublist]
    y = np.array(y)
    y_time = [item for sublist in seqYtime for item in sublist]
    y_time = np.array(y_time)
    x = x.reshape(x.shape[0], x.shape[2], x.shape[3])
    print(x.shape, y.shape, bmi.shape)
    return x, y, y_time, bmi
#edef

def to_sequences_data(SEQUENCE_SIZE, obs, yobs, part_level_data, downsample_rate):
######
# for every EEm value: 
# 1)take its timestamp, 
# 2)take all the accel timestamps, before that one, 
# 3)keep the last accel timestamps(accel instances) in the size of the window given
    # SEQUENCE_SIZE: the length of the sequences to be build
    # obs: accel predictors
    # yobs: target (EEm)
    # part_level_data: participants' level data
    # downsample_rate: the sampling rate to downsample the accelerometer data for the sequences    
######

    x, y, y_time = [], [], []
    for i in yobs.index.tolist():
        # keep the window of data to sequence
        window = obs.loc[:i].iloc[-SEQUENCE_SIZE:]
        windowInMinutes = round(SEQUENCE_SIZE/(downsample_rate*60))  
        if windowInMinutes < 1:
            if window.shape[0]==0:
                continue
            else:
                windowInSeconds = round((SEQUENCE_SIZE/(downsample_rate*60))*60)  
    #             print('Window smaller than a minute, equal to: '+ str(SEQUENCE_SIZE/(downsample_rate*60))+' in sec: '+str(windowInSeconds))       
                windowSize = (window.index[-1]-window.index[0])/np.timedelta64(1,'s') #calculate window size in seconds
                if window.shape[0] == SEQUENCE_SIZE :
                    after_window = yobs[i]
                    x.append(window)
                    y.append(after_window)
                    y_time.append(i)

                #fi
            #fi

        else:
            if window.shape[0]==0:
                continue
            else:
                windowSize = (window.index[-1]-window.index[0])/np.timedelta64(1,'m') #calculate window size in minutes
                window = window.values.reshape(-1, window.shape[0], window.shape[1])
                if window.shape[1] == SEQUENCE_SIZE and windowSize < windowInMinutes:
                    after_window = yobs[i]
                    x.append(window)
                    y.append(after_window)
                    y_time.append(i)
                #fi
            #fi
        #fi

    # BMI here are the demographics data and y is the number of sequences (not the length of a seq)
    BMI = np.empty((len(y), np.array(part_level_data).shape[1]))
    BMI = part_level_data*len(y)
    return x, y, BMI, y_time
#edef

In [ ]:
# # # # # # # # # # # # FIXED VARs # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
numberForVal = 1 # the number of participants you want per group as validation set (random seed is the number of participants' id)
SEQUENCE_SIZE = 50 # fixed size of esquences
windowSize = 2 # in minutes
func = 'std'
# # # # # # # # # # # # GIVEN VARS # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# downsample_rate = 0.666666 # sampling rate of predictors given in Hz (2Hz create a window 4min, 4Hz -> 2min etc. )
EEm_rate = 0 # sampling rate of target in seconds e.g. EEm=10 means one input every 10 seconds

downsample_rate = round(SEQUENCE_SIZE/(windowSize*60),3)

print('downsample_rate is', downsample_rate, 'Hz')
# windowSize = round(SEQUENCE_SIZE/(downsample_rate*60),3) # size of window calculated from given downsample_rate 
print('windowSize is', windowSize, 'minutes')
if windowSize < 1:
    save_dir = '/preprocessedData/sequences/'+str(int(round(windowSize*60,0)))+'secs_seqs_'+str(numberForVal)+'_EEm_'+str(EEm_rate)+'ds_'+func+'_'+str(SEQUENCE_SIZE)+'/'
else:
    save_dir = '/preprocessedData/sequences/'+str(int(round(windowSize,0)))+'min_seqs_'+str(numberForVal)+'_EEm_'+str(EEm_rate)+'ds_'+func+'_'+str(SEQUENCE_SIZE)+'/'
print('Save directory:', save_dir)

In [ ]:
# calling the functions above to read data, standarize them, build seqs and save files
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

print('===========================================')

# read and concat all participants data
df_gene = read_data()

for i in range(2,37):
    if i in [2,3,4,19]:
        continue
    else: 
        if len(str(i)) == 1:
            name = 'GOTOV0'+str(i)
        else: 
            name = 'GOTOV'+str(i)
    print('Build sequences for test patient', name)
    # select participant data for df_train, df_val, df_test 
    df_train, df_val, df_test = select_Data(name, randomSeed, numberForVal, df_gene_st)        
    standardizing_data(name, df_train, df_val, df_test, SEQUENCE_SIZE, downsample_rate, func, EEm_rate, save_dir)
    print('===========================================')